In [1]:
import os
from tqdm import tqdm
from train import parse_and_preprocess_data, preprocess_for_training
from predict import ModelPrediction
from datetime import datetime
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from scipy.interpolate import griddata

# Load configuration from environment variables
symbol = os.getenv('SYMBOL')
interval = int(os.getenv('INTERVAL'))
tss_n_splits = int(os.getenv('TSS_N_SPLITS'))
n_back_features = int(os.getenv('N_BACK_FEATURES'))
tss_test_size = int(os.getenv('TSS_TEST_SIZE'))

os.listdir('../weis/')

['cb']

### Parse backtest data

In [33]:
parsing_params = {
    'category': 'linear',
    'symbol': symbol,
    'interval': interval,
    'testnet': False,
    'start_date': datetime(2024, 8, 10),
    'end_date': datetime.now()
}

backtest_data_dict = parse_and_preprocess_data(parsing_params, False)

(1760, 196)


### Prediction

In [34]:
prediction_model = ModelPrediction(
        model_directory='../weis/',
        verbose=False
)

# Initialize lists to store aggregated probabilities
buy_probas, hold_probas, sell_probas = [], [], []

# Loop through each dataframe and corresponding model to get predictions
for key, backtest_data in backtest_data_dict.items():
    _, _, _, X, _ = preprocess_for_training(backtest_data)
    backtest_data = backtest_data.iloc[-len(X):].reset_index(drop=True)

    # Get predictions
    buy_proba, hold_proba, sell_proba = prediction_model.predict(X)
    buy_probas.append(buy_proba)
    hold_probas.append(hold_proba)
    sell_probas.append(sell_proba)

# Average the probabilities across all models
buy_probas = np.mean(buy_probas, axis=0)
sell_probas = np.mean(sell_probas, axis=0)
hold_probas = np.mean(hold_probas, axis=0)

# Combine the probabilities into a list of tuples for further processing
preds = list(zip(buy_probas, hold_probas, sell_probas))

### Make predictions and decisions

In [42]:
ts = [0.67, 0, 0.67]

def make_decision(x, ts): 
    b, h, s = x
    if b > ts[0]:
        return 0
    elif s > ts[2]:
        return 2
    else:
        return 1

decisions = [make_decision(x, ts) for x in preds]

### Backtest the strategy

In [ ]:
def backtest_strategy(decisions, initial_balance=10000, price_data=None, taker_fee=0.001, maker_fee=0.001):
    balance = initial_balance
    position = 0  # Current position: +1 for holding, 0 for no position
    transaction_log = []  # To keep track of all transactions

    for i, decision in enumerate(decisions):
        price = price_data[i] if price_data is not None else 100  # Assume a default price if not provided
        
        if decision == 0:  # Buy
            if position == 0:  # Only buy if not holding
                position = (balance / price) * (1 - taker_fee)  # Buy as much as possible minus fees
                balance = 0  # Spend all balance
                transaction_log.append(f"Buy at {price} with {taker_fee*100}% fee")
        elif decision == 2:  # Sell
            if position > 0:  # Only sell if holding
                balance = position * price * (1 - taker_fee)  # Sell all holdings minus fees
                position = 0  # Clear position
                transaction_log.append(f"Sell at {price} with {taker_fee*100}% fee")
        # No action needed for hold (1)

    # If still holding at the end, sell at the last price
    if position > 0:
        balance = position * price * (1 - taker_fee)  # Sell all at the last known price minus fees
        transaction_log.append(f"Final sell at {price} with {taker_fee*100}% fee")

    final_balance = balance
    roi = (final_balance - initial_balance) / initial_balance * 100

    return final_balance, roi

initial_balance = 100

# Run the backtest
final_balance, roi = backtest_strategy(decisions, initial_balance, backtest_data['CLOSE'])

print(roi)

26.786494596342663


### Grid searching the best thresholds

In [ ]:
all_tss, all_rois, all_tss_1 = [], [], []

triplets = []

for thresh in tqdm(np.linspace(0.51, 0.9, 70)):
    for thresh_1 in np.linspace(0.51, 0.9, 70):
        tss = [thresh, 0, thresh_1]
    
        gs_decisions = [make_decision(x, tss) for x in preds]
    
        final_balance, roi = backtest_strategy(gs_decisions, initial_balance, backtest_data['CLOSE'])
    
        triplets.append([thresh, thresh_1, roi])
        
index = max(range(len(triplets)), key=lambda i: triplets[i][2])
triplets[index]

In [ ]:
# Convert triplets to a DataFrame
df = pd.DataFrame(triplets, columns=['x', 'y', 'metrics'])

# Create a grid
grid_x, grid_y = np.mgrid[
    df['x'].min():df['x'].max():100j, 
    df['y'].min():df['y'].max():100j
]

# Interpolate the metrics onto the grid
grid_z = griddata(
    (df['x'], df['y']), df['metrics'], 
    (grid_x, grid_y), method='cubic'
)

# Create the heatmap using Plotly
fig = go.Figure(data=go.Heatmap(
    z=grid_z.T,  # Transpose to align with x and y axes
    x=np.linspace(df['x'].min(), df['x'].max(), 100), 
    y=np.linspace(df['y'].min(), df['y'].max(), 100),
    colorscale='Viridis'
))

fig.update_layout(
    title='Interpolated Heatmap of Metrics',
    xaxis_title='X axis',
    yaxis_title='Y axis',
    width=800,
    height=800,
    coloraxis_colorbar=dict(title="Metrics")
)

# Show the figure
fig.show()

In [ ]:
def test_multiple_days(n_days_from_now):
    for i in range(-1, n_days_from_now-1):

        
        now = datetime.now()
        today = now.day
        tomonth = now.month

        print(f'Results for 2024-{tomonth}-{today-i-1}  --  2024-{tomonth}-{today-i}')
        
        parsing_params = {
            'category': 'linear',
            'symbol': symbol,
            'interval': interval,
            'testnet': False,
            'start_date': datetime(2024, tomonth, today-i-1),
            'end_date': datetime(2024, tomonth, today-i)
        }
        
        backtest_data = parse_and_preprocess_data(parsing_params, False)
        _, _, _, X, _ = preprocess_for_training(backtest_data)
        backtest_data = backtest_data.iloc[-len(X):].reset_index(drop=True)

        prediction_model = ModelPrediction(
            model_path='../weis/cb',
            verbose=False
        ).model
        
        preds = prediction_model.predict_proba(X)
    
        all_tss, all_rois, all_tss_1 = [], [], []
    
        triplets = []
        
        for thresh in tqdm(np.linspace(0.51, 0.9, 30)):
            for thresh_1 in np.linspace(0.51, 0.9, 30):
                tss = [thresh, 0, thresh_1]
            
                gs_decisions = [make_decision(x, tss) for x in preds]
            
                final_balance, roi = backtest_strategy(gs_decisions, initial_balance, backtest_data['CLOSE'])
            
                triplets.append([thresh, thresh_1, roi])
    
        # Convert triplets to a DataFrame
        df = pd.DataFrame(triplets, columns=['x', 'y', 'metrics'])
        
        # Create a grid
        grid_x, grid_y = np.mgrid[
            df['x'].min():df['x'].max():100j, 
            df['y'].min():df['y'].max():100j
        ]
        
        # Interpolate the metrics onto the grid
        grid_z = griddata(
            (df['x'], df['y']), df['metrics'], 
            (grid_x, grid_y), method='cubic'
        )
        
        # Create the heatmap using Plotly
        fig = go.Figure(data=go.Heatmap(
            z=grid_z.T,  # Transpose to align with x and y axes
            x=np.linspace(df['x'].min(), df['x'].max(), 100), 
            y=np.linspace(df['y'].min(), df['y'].max(), 100),
            colorscale='Viridis'
        ))
        
        fig.update_layout(
            title='Interpolated Heatmap of Metrics',
            xaxis_title='X axis',
            yaxis_title='Y axis',
            width=800,
            height=800,
            coloraxis_colorbar=dict(title="Metrics")
        )
        
        # Show the figure
        fig.show()
        
test_multiple_days(20)